In [4]:
import os
import time

import cv2
import model
import numpy as np
import torch
import torch.optim
import torchvision
from PIL import Image


def lowlight(data_lowlight):
    start = time.time()
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    scale_factor = 12

    data_lowlight = data_lowlight.convert('RGB')

    data_lowlight = np.asarray(data_lowlight) / 255.0

    data_lowlight = torch.from_numpy(data_lowlight).float()

    h = (data_lowlight.shape[0] // scale_factor) * scale_factor
    w = (data_lowlight.shape[1] // scale_factor) * scale_factor
    data_lowlight = data_lowlight[0:h, 0:w, :]
    data_lowlight = data_lowlight.permute(2, 0, 1)
    data_lowlight = data_lowlight.cuda().unsqueeze(0)

    DCE_net = model.enhance_net_nopool(scale_factor).cuda()
    DCE_net.load_state_dict(torch.load('snapshots_Zero_DCE++/Epoch99.pth'))
    start = time.time()
    enhanced_image, params_maps = DCE_net(data_lowlight)

    end = time.time()
    infer_time = end - start
    print(f'infer_time: {infer_time}')
    return enhanced_image

In [5]:
test_low_light_images = os.listdir(
    '/home/hiepdvh/low-light-recognition-trial/Zero-DCE++/lol_dataset/eval15/low'
)

for low_light_image in test_low_light_images:
    low_light_image_path = os.path.join(
        '/home/hiepdvh/low-light-recognition-trial/Zero-DCE++/lol_dataset/eval15/low',
        low_light_image,
    )
    original_image = Image.open(low_light_image_path)
    enhanced_image = lowlight(original_image)
    result_path = os.path.join(
        '/home/hiepdvh/low-light-recognition-trial/Zero-DCE++/lol_dataset/eval15/high',
        low_light_image,
    )
    torchvision.utils.save_image(enhanced_image, result_path)

infer_time: 0.0005617141723632812
infer_time: 0.0004975795745849609
infer_time: 0.00049591064453125
infer_time: 0.0005018711090087891
infer_time: 0.0004944801330566406
infer_time: 0.0005142688751220703
infer_time: 0.0005056858062744141
infer_time: 0.0004894733428955078
infer_time: 0.0008141994476318359
infer_time: 0.0005934238433837891
infer_time: 0.0005054473876953125
infer_time: 0.0005085468292236328
infer_time: 0.0004856586456298828
infer_time: 0.0005266666412353516
infer_time: 0.0005385875701904297


In [6]:
import os

import cv2
import numpy as np
import torch
from lpips import LPIPS
from scipy.stats import entropy
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


def calculate_metrics(original_img, enhanced_img):
    # Chuyển đổi ảnh từ [0, 255] sang [0, 1]
    original_img = original_img.astype(np.float32) / 255.0
    enhanced_img = enhanced_img.astype(np.float32) / 255.0
    original_img.resize(enhanced_img.shape)
    # PSNR
    psnr_value = psnr(original_img, enhanced_img, data_range=1.0)

    # SSIM
    ssim_value, _ = ssim(original_img, enhanced_img, full=True, data_range=1.0)

    # Entropy
    entropy_original = entropy(original_img.flatten())
    entropy_enhanced = entropy(enhanced_img.flatten())

    # Standard Deviation
    std_dev_original = np.std(original_img)
    std_dev_enhanced = np.std(enhanced_img)
    original_tensor = torch.from_numpy(original_img)
    enhanced_tensor = torch.from_numpy(enhanced_img)
    lpips_value = lpips_model(original_tensor, enhanced_tensor).item()
    return (
        psnr_value,
        ssim_value,
        entropy_original,
        entropy_enhanced,
        std_dev_original,
        std_dev_enhanced,
        lpips_value,
    )

In [7]:
lpips_model = LPIPS(net='vgg')
original_image_folder = (
    '/home/hiepdvh/low-light-recognition-trial/Zero-DCE++/lol_dataset/eval15/high'
)
enhanced_image_folder = (
    '/home/hiepdvh/low-light-recognition-trial/Zero-DCE++/lol_dataset/eval15/low'
)

images = os.listdir(original_image_folder)
results = ''
average_psnr = 0
average_ssim = 0
average_entropy = 0
average_std = 0
average_lpips = 0
for i, image in enumerate(images):
    original_image_path = os.path.join(original_image_folder, image)
    enhanced_image_path = os.path.join(enhanced_image_folder, image)
    original_img = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
    enhanced_img = cv2.imread(enhanced_image_path, cv2.IMREAD_GRAYSCALE)

    (
        psnr_value,
        ssim_value,
        entropy_original,
        entropy_enhanced,
        std_dev_original,
        std_dev_enhanced,
        lpips_value,
    ) = calculate_metrics(original_img, enhanced_img)
    result = f'{image}\npsnr: {psnr_value}, ssim: {ssim_value}, entropy: {entropy_enhanced}, std: {std_dev_enhanced}, lpips: {lpips_value}'
    results = results + result + '\n'
    average_psnr = (average_psnr*i + psnr_value)/(i+1)
    average_ssim = (average_ssim*i + ssim_value)/(i+1)
    average_entropy = (average_entropy*i + entropy_enhanced)/(i+1)
    average_std= (average_std*i + std_dev_enhanced)/(i+1)
    average_lpips = (average_lpips*i + lpips_value)/(i+1)

print(results)
print(f' average_psnr: {average_psnr} \n average_ssim: {average_ssim} \n average_entropy: {average_entropy} \n average_std: {average_std} \n average_lpips: {average_lpips}')

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/hiepdvh/miniconda3/envs/livefaceidapp/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hiepdvh/miniconda3/envs/livefaceidapp/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/hiepdvh/miniconda3/envs/livefaceidapp/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth
665.png
psnr: 13.839268922802699, ssim: 0.18199984216234696, entropy: 12.1690034866333, std: 0.019163131713867188, lpips: 0.41301247477531433
22.png
psnr: 11.099410584086474, ssim: 0.30083373389214335, entropy: 12.101876258850098, std: 0.0507991686463356, lpips: 0.31196820735931396
778.png
psnr: 14.5348520015447, ssim: 0.16601431221509239, entropy: 11.859112739562988, std: 0.02995860017836094, lpips: 0.3974076509475708
780.png
psnr: 13.753789428944948, ssim: 0.20230139691189927, entropy: 11.887100219726562, std: 0.05470995232462883, lpips: 0.4016571044921875
111.png
psnr: 10.742132299604938, ssim: 0.2986297827913984, entropy: 12.224459648132324, std: 0.042161352932453156, lpips: 0.290187269449234
146.png
psnr: 10.371230804369118, ssim: 0.3546604842240138, entropy: 12.196784973144531, std: 0.04989078640937805, lpips: 0.2696795165538788
493.png
psnr: 15.24459806501999, s